In [1]:
import pandas
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [3]:
data = pandas.read_json("https://vpic.nhtsa.dot.gov/api/vehicles/GetMakesForVehicleType/car?format=json")
makes = pandas.io.json.json_normalize(data['Results'])

In [4]:
t = pandas.DataFrame(columns=['Year', 'Make_ID', 'Make_Name', 'Model_ID', 'Model_Name'])

In [ ]:
for make in makes.iterrows():    
    print(make[1]['MakeName'])
    make_id = make[1]['MakeId']
    for year in range(2000,2021):
        r = requests_retry_session().get("https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeIdYear/makeId/{}/modelyear/{}/vehicletype/car?format=json".format(make_id,year))
        data = pandas.read_json(r.text)
        models = pandas.DataFrame(pandas.io.json.json_normalize(data['Results']), columns=['Make_ID', 'Make_Name', 'Model_ID', 'Model_Name'])
        models['Year'] = year
        t = t.append(models, ignore_index=True, sort=False)

Aston Martin
Tesla
Jaguar
Maserati
Rolls Royce
Toyota
Mercedes-Benz
BMW
Bugatti
MINI
Ford
Lincoln
Mercury
Lotus
Chevrolet
Buick
Cadillac
Holden
Opel
GMC
Mazda
Honda
Acura
Dodge
Chrysler
Nissan
Infiniti
Mitsubishi
Volkswagen
Volvo
Fiat
Alfa Romeo
Lancia
Hyundai
Kia
Lamborghini
smart
Suzuki
Lexus
Subaru
Maybach
Pontiac
Morgan
Isuzu
SAAB
Audi
Bentley
Porsche
Ferrari
AM General
Creative Coachworks
D & N Trailers
AC Propulsion
Daihatsu
Falcon
EV Innovations
Faw Jiaxing Happy Messenger
FISKER AUTOMOTIVE
Nina
Saturn
Daewoo
American Motors
Formula 1 Street Com
Geo
FortuneSport VES
AAS
EQUUS AUTOMOTIVE
Electric Mobile Cars
Avera Motors
GREENTECH AUTOMOTIVE
Bakkura Mobility
Th!nk
COMMUNITY CARS 
CONTEMPORARY CLASSIC CARS
Koenigsegg
BYD
Kepler Motors
McLaren
mycar
Eagle
Plymouth
Nanchang Freedom Technology Limited Company 


In [12]:
t.to_csv('makeid_by_year.csv',index=False)